<a href="https://colab.research.google.com/github/MattyXarope/Comp-Sci-Homework-3-/blob/main/HypernymExtractor%20SemiFinal%20-1/3/2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Packages**

In [1277]:
# Import libraries required
import re
import nltk
import itertools
import pandas as pd
from nltk.tag.perceptron import PerceptronTagger
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# **Hearst Hyponym Extractor and Preprocessing**

## Hearst Patterns

In [1208]:
# Hearst Patterns
hearst_patterns=[ (
                '(NP_\\w+ (, )?such as (NP_\\w+ ?(, )?(and |or )?)+)',
                'first'
            ),
            (
                '(such NP_\\w+ (, )?as (NP_\\w+ ?(, )?(and |or )?)+)',
                'first'
            ),
            (
                '((NP_\\w+ ?(, )?)+(and |or )?other NP_\\w+)',
                'last'
            ),
            (
                '(NP_\\w+ (, )?include (NP_\\w+ ?(, )?(and |or )?)+)',
                'first'
            ),
            (
                '(NP_\\w+ (, )?especially (NP_\\w+ ?(, )?(and |or )?)+)',
                'first'
            ),
            (
                    '((NP_\\w+ ?(, )?)+(and |or )?any other NP_\\w+)',
                    'last'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and |or )?some other NP_\\w+)',
                    'last'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and |or )?be a NP_\\w+)',
                    'last'
                ),
                (
                    '(NP_\\w+ (, )?like (NP_\\w+ ? (, )?(and |or )?)+)',
                    'first'
                ),
                (
                    'such (NP_\\w+ (, )?as (NP_\\w+ ? (, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and |or )?like other NP_\\w+)',
                    'last'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and |or )?one of the NP_\\w+)',
                    'last'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and |or )?one of these NP_\\w+)',
                    'last'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and |or )?one of those NP_\\w+)',
                    'last'
                ),
                (
                    'example of (NP_\\w+ (, )?be (NP_\\w+ ? '
                    '(, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and |or )?be example of NP_\\w+)',
                    'last'
                ),
                (
                    '(NP_\\w+ (, )?for example (, )?'
                    '(NP_\\w+ ?(, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and |or )?which be call NP_\\w+)',
                    'last'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and |or )?which be name NP_\\w+)',
                    'last'
                ),
                (
                    '(NP_\\w+ (, )?mainly (NP_\\w+ ? (, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '(NP_\\w+ (, )?mostly (NP_\\w+ ? (, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '(NP_\\w+ (, )?notably (NP_\\w+ ? (, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '(NP_\\w+ (, )?particularly (NP_\\w+ ? '
                    '(, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '(NP_\\w+ (, )?principally (NP_\\w+ ? (, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '(NP_\\w+ (, )?in particular (NP_\\w+ ? '
                    '(, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '(NP_\\w+ (, )?except (NP_\\w+ ? (, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '(NP_\\w+ (, )?other than (NP_\\w+ ? (, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '(NP_\\w+ (, )?e.g. (, )?(NP_\\w+ ? (, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '(NP_\\w+ \\( (e.g.|i.e.) (, )?(NP_\\w+ ? (, )?(and |or )?)+'
                    '(\\. )?\\))',
                    'first'
                ),
                (
                    '(NP_\\w+ (, )?i.e. (, )?(NP_\\w+ ? (, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and|or)? a kind of NP_\\w+)',
                    'last'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and|or)? kind of NP_\\w+)',
                    'last'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and|or)? form of NP_\\w+)',
                    'last'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and |or )?which look like NP_\\w+)',
                    'last'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and |or )?which sound like NP_\\w+)',
                    'last'
                ),
                (
                    '(NP_\\w+ (, )?which be similar to (NP_\\w+ ? '
                    '(, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '(NP_\\w+ (, )?example of this be (NP_\\w+ ? '
                    '(, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '(NP_\\w+ (, )?type (NP_\\w+ ? (, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and |or )? NP_\\w+ type)',
                    'last'
                ),
                (
                    '(NP_\\w+ (, )?whether (NP_\\w+ ? (, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '(compare (NP_\\w+ ?(, )?)+(and |or )?with NP_\\w+)',
                    'last'
                ),
                (
                    '(NP_\\w+ (, )?compare to (NP_\\w+ ? (, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '(NP_\\w+ (, )?among -PRON- (NP_\\w+ ? '
                    '(, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and |or )?as NP_\\w+)',
                    'last'
                ),
                (
                    '(NP_\\w+ (, )? (NP_\\w+ ? (, )?(and |or )?)+ '
                    'for instance)',
                    'first'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and|or)? sort of NP_\\w+)',
                    'last'
                ),
                (
                    '(NP_\\w+ (, )?which may include (NP_\\w+ '
                    '?(, )?(and |or )?)+)',
                    'first'
                ),]

## Text Preprocessing

In [1209]:
# Noun phrase chunking
chunk_patterns=r'''NP:{<DT>?<JJ.*>*<NN.*>+}
                      {<NN.*>+}
                '''
#chunk parser
nounphrase_chunker=nltk.RegexpParser(chunk_patterns)

# Part of Speech tagging
pos_tagger=PerceptronTagger()

# Text Preprocessing
def prepare(raw_text):
    sentences=nltk.sent_tokenize(raw_text.strip())
    sentences=[nltk.word_tokenize(sent) for sent in sentences]
    sentences=[pos_tagger.tag(sent) for sent in sentences]
    return sentences

def chunk(raw_text):
    sentences=prepare(raw_text.strip())
    all_chunks=[]
    for sentence in sentences:
        chunks=nounphrase_chunker.parse(sentence)
        all_chunks.append(prepare_chunks(chunks))
    all_sentences=[]
    for raw_sentence in all_chunks:
        sentence = re.sub(r"(NP_\w+ NP_\w+)+",lambda m: m.expand(r'\1').replace(" NP_", "_"),raw_sentence)
        all_sentences.append(sentence)
    return all_sentences

# Method to replace occurrences in text that are noun phrases and make them start with NP_<token> or just keep it as is
def prepare_chunks(chunks):
    terms=[]
    for chunk in chunks:
        label=None
        try:
            label=chunk.label()
        except:
            pass
        if label is None:
            token=chunk[0]
            terms.append(token)
        else:
            np='NP_'+'_'.join([a[0] for a in chunk])
            terms.append(np)
    return ' '.join(terms)

def remove_np_term(term):
    return term.replace('NP_','').replace('_',' ')

## Extract Hyponyms from Preprocessed Text

In [1210]:
def find_hyponyms(rawtext):
    hypo_hypernyms=[]
    np_tagged_sentences=chunk(rawtext)

    for sentence in np_tagged_sentences:
        for (hearst_pattern,parser) in hearst_patterns:
            matches=re.search(hearst_pattern,sentence)
            if matches:
                match_str=matches.group(0)
                nps=[a for a in match_str.split() if a.startswith('NP_')]

                if parser=='first':
                    hypernym=nps[0]
                    hyponyms=nps[1:]
                else:
                    hypernym=nps[-1]
                    hyponyms=nps[:-1]
                for i in range(len(hyponyms)):
                    hypo_hypernyms.append((remove_np_term(hyponyms[i]),remove_np_term(hypernym)))
    return hypo_hypernyms

## Function to Apply Hearst Patterns to Data

In [1211]:
# Apply Hearst Patterns
def extractHearstWikipedia(input):
    extractions=[]
    lines_processed=0

    with open(input,'r', encoding='latin-1') as f:
        for  line in f:
            lines_processed+=1
            line=line.strip()
            if not line:
                continue
            line_split=line.split('\t')
            sentence=line_split[0]
            hypo_hyper_pairs=find_hyponyms(sentence)
            extractions.append(hypo_hyper_pairs)

            if lines_processed%1000==0:
                print('Lines Processed: {}'.format(lines_processed))

    extractions=[x for x in extractions if x!=[]]
    extractions=list(itertools.chain.from_iterable(extractions))

#Creates file 'subset_hypernyms' with the pairs      
    with open('subset_hypernyms','w') as f:
        for pair in extractions:
            try:
                f.write(str(pair)+'\n')
            except ValueError:
                pass

    return extractions

# **Applying Hearst Function to our Test Data**

## Set Path for our Data File and Apply Hearst Function

In [1212]:
#Set path for our test data file 
path = "/content/English Text Raw (01).txt"

#Applies Hearst function to our data
hypo_hyper_pairs=extractHearstWikipedia(path)

Lines Processed: 6000
Lines Processed: 7000
Lines Processed: 8000
Lines Processed: 10000
Lines Processed: 11000
Lines Processed: 12000
Lines Processed: 13000
Lines Processed: 14000
Lines Processed: 16000
Lines Processed: 17000
Lines Processed: 18000
Lines Processed: 20000
Lines Processed: 21000
Lines Processed: 22000
Lines Processed: 25000
Lines Processed: 29000
Lines Processed: 32000
Lines Processed: 36000
Lines Processed: 37000
Lines Processed: 39000
Lines Processed: 40000
Lines Processed: 42000
Lines Processed: 43000
Lines Processed: 46000
Lines Processed: 47000
Lines Processed: 48000
Lines Processed: 49000
Lines Processed: 50000
Lines Processed: 51000
Lines Processed: 52000
Lines Processed: 55000
Lines Processed: 57000
Lines Processed: 58000
Lines Processed: 62000
Lines Processed: 63000
Lines Processed: 64000
Lines Processed: 65000
Lines Processed: 67000
Lines Processed: 69000
Lines Processed: 70000
Lines Processed: 71000
Lines Processed: 72000
Lines Processed: 73000
Lines Processe

## Postprocessing the Data Retrieved

In [1213]:
#Create lists from the hyponyms-hypernyms extracted from our data
x = hypo_hyper_pairs
hyponyms, hypernyms = zip(*x)

#Function to reduce noun phrases down to their heads and lemmatizes
from nltk.stem.wordnet import WordNetLemmatizer
lemma=WordNetLemmatizer()
def post_process(text):
    text=text.split()
    length=len(text)
    return lemma.lemmatize(text[length-1])

#Apply post_process function to hyponyms and hypernyms
hyponyms_new=[]
for i in hyponyms:
  hyponyms_new.append(post_process(i.lower()))

hypernyms_new=[]  
for i in hypernyms:
  hypernyms_new.append(post_process(i.lower()))  

#Create dictionary from post processed hyponyms and hypernyms
full_list = zip(hyponyms_new, hypernyms_new)
hypohyper_hearst_patterns = dict(full_list)
   

## Print our Postprocessed Hypo-Hyper Pairs from Hearst Data

In [1214]:
#Prints the postprocessed hypo-hyper pairs 
hypohyper_hearst_patterns

{'time': 'chairman',
 'name': 'adult',
 'macaulay': 'hallam',
 'shakespeare': 'poet',
 'grotius': 'name',
 'bacon': 'name',
 'hobbes': 'feature',
 'university': 'equivalent',
 'academy': 'programme',
 'philosophy': 'whole',
 'woodlot': 'facility',
 'course': 'association',
 '30': 'facility',
 'redding': 'product',
 'rusk': 'product',
 'product': 'thermostat',
 'record': 'number',
 'stamford': 'center',
 'cimbalom': 'instrument',
 'montessori': 'educator',
 'steiner': 'pioneer',
 'tolstoy': 'anarchist',
 'guardia': 'anarchist',
 'education': 'jagirdars',
 'holt': 'critic',
 'goodman': 'santa',
 'mayer': 'result',
 'illich': 'critic',
 'kohl': 'educator',
 'kozol': 'educator',
 'school': 'child',
 'tarot': 'repository',
 'ability': 'presenter',
 'system': 'body',
 'class': 'group',
 'knowledge': 'action',
 'neal': 'robert',
 'savage': 'vera',
 'drake': 'stulwitz',
 'macdonald': 'jr',
 'ryan': 'herself',
 'howard': 'holly',
 'gleason': 'joe',
 'yellow': 'infection',
 'catapult': 'trail',


# **Comparison of our Data with Task Data** 

## Importing and Processing Task Data 

In [1215]:
# Import Task Training Data (Hyponyms)
with open('/content/1A.english.training.data.txt','r') as f:
    inputlines=f.read().strip().split('\n')
hyponyms_task_training = inputlines

# Import Task Gold Training Data (Hypernyms)
with open('/content/1A.english.training.gold.txt','r') as f:
    inputlines2=f.read().strip().split('\n')
hypernyms_gold_task_training = []
for line in inputlines2:
    split=line.split('\t')
    hypo=split[0:]
    hypernyms_gold_task_training.append(hypo)

#Create dictionary from training data hyponyms and hypernyms
taskdata = zip(hyponyms_task_training, hypernyms_gold_task_training)
hypohyper_training_data = dict(taskdata)

## Print  Postprocessed Hypo-Hyper Pairs from Training Data

In [1216]:
#Prints the postprocessed hypo-hyper pairs 
print(hypohyper_training_data)

{'blackfly': ['homopterous insect', 'insect'], 'Turonian': ['technical specification', 'geologic timescale', 'physical property', 'geological period', 'magnitude', 'unit of time', 'geological time', 'geologic time'], 'abhorrence': ['distaste', 'hatred', 'hate', 'disgust'], 'tropical storm': ['atmosphere', 'windstorm', 'violent storm', 'air current', 'atmospheric state', 'density', 'current of air', 'storm damage', 'atmospheric phenomenon', 'storm', 'cyclone', 'natural phenomenon', 'tempest', 'wind'], 'militarization': ['social control'], 'pollution': ['dirtiness', 'dirtying', 'environmental condition', 'impurity', 'sanitary condition', 'uncleanness'], 'photomicrograph': ['picture', 'photograph', 'photo', 'image', 'pic', 'digital image'], 'swamp gum': ['eucalyptus', 'plant', 'eucalyptus tree', 'angiosperm', 'gum', 'gum tree', 'woody plant', 'eucalypt'], 'song': ['possession', 'sound', 'work of art', 'musical composition', 'vocal music', 'musical work', 'piece of music', 'human language'

## Comparing our Hearst Data and Task Training Data

In [1282]:
# "hypohyper_hearst_patterns" is our hearst data
# "hypohyper_training_data" is the training data

#get hyponyms that the training and data have in common
intersect = []
for item in hypohyper_hearst_patterns.keys():
  if item in hypohyper_training_data.keys():
    intersect.append(item)

#Print the hyponyms in common
hypo_in_common = len(intersect)
print(f'There are {hypo_in_common} hyponyms in common between our corpus and the training data.')
#print(f'These are the hyponyms in common {intersect}')

#Get the hypernyms for our data from the list of hyponyms in common
hypernyms_for_hearst_in_common = [hypohyper_hearst_patterns[value] for value in intersect]
#print(f'These are the hypernyms for those hyponyms in common: {hypernyms_for_hearst_in_common}')

#Create dictionary from the two above lists
combined_hyp_hyper = zip(intersect, hypernyms_for_hearst_in_common)
combined_hyp_hyper_dict = dict(combined_hyp_hyper)

#Prints a csv file with the number in common between the two lists using Hearst data 
df = pd.DataFrame.from_dict(combined_hyp_hyper_dict, orient="index")
df.to_csv("HearstCommon.csv")

#This gets the keys and values from hypohyper_training_data for the ones in common 
dict1_cond = {k:v for (k,v) in hypohyper_training_data.items() if k in intersect}
#print(dict1_cond)

#Prints a csv file with the number in common between the two lists using Training data 
df = pd.DataFrame.from_dict(dict1_cond, orient="index")
df.to_csv("TrainingCommon.csv")

#Find the shared values
shared_items = {k: hypohyper_hearst_patterns[k] for k in hypohyper_hearst_patterns if k in hypohyper_training_data and hypohyper_hearst_patterns[k] in hypohyper_training_data[k]}

#Prints a csv file with hyponyms with their hypernyms - this is the final data that we want
df = pd.DataFrame.from_dict(shared_items, orient="index")
df.to_csv("SharedHypoHyper.csv")

There are 176 hyponyms in common between our corpus and the training data.
There are 11 items that share the same hyponym-hypernym between the Hearst data and the Training data.
They are: {'cat': 'animal', 'silver': 'metal', 'referee': 'person', 'bird': 'animal', 'crab': 'crustacean', 'chronic': 'disease', 'bean': 'vegetable', 'fine': 'penalty', 'copper': 'metal', 'cricket': 'competition', 'fish': 'animal'}


## Print Hyponym-Hypernym Pairs in Common Between Hearst and Task Data Sets

In [1283]:
#Prints info about the shared hypo-hypernym pairs between data lists
print(f'There are {len(shared_items)} items that share the same hyponym-hypernym between the Hearst data and the Training data.')
print(f'They are: {shared_items}')  

There are 11 items that share the same hyponym-hypernym between the Hearst data and the Training data.
They are: {'cat': 'animal', 'silver': 'metal', 'referee': 'person', 'bird': 'animal', 'crab': 'crustacean', 'chronic': 'disease', 'bean': 'vegetable', 'fine': 'penalty', 'copper': 'metal', 'cricket': 'competition', 'fish': 'animal'}


## Error Analysis of Data

In [1219]:
# # function to read in from truth values and the predictions on trainset
# def read_pred(file):
#     labels=[]
#     with open(file,'r') as f:
#         data=f.read().strip().split('\n')
#     for line in data:
#         hypo,hyper,label=line.split('\t')
#         if label=='True':
#             labels.append(1)
#         else:
#             labels.append(0)

#     return labels
# truth_values=read_pred('bless2011/data_lex_train.tsv')
# predicted_values=read_pred('train_pred.tsv')

# # Precision, Recall and F1 Score
# from sklearn.metrics import classification_report
# classification_report(truth_values,predicted_values)
# from sklearn.metrics import precision_score
# from sklearn.metrics import recall_score
# from sklearn.metrics import f1_score
# print('BLESS 2011 Data \n')
# print('Precision: {}'.format(precision_score(truth_values,predicted_values)))
# print('Recall: {}'.format(recall_score(truth_values,predicted_values)))
# print('F1 Score: {}'.format(f1_score(truth_values,predicted_values)))